# Chapter 9: Classes (part 7)

## Single Inheritance

- This is the last version of `Car` from the previous section.
- There is a problem with this class that will not be apparent until we create subclasses.

In [ ]:
class Car:
    """
       This is the base class
    """

    __count = 0

    def __init__(self, name = None):
        self.__set_name(name)
        type(self).__count += 1

    def __del__(self):
        type(self).__count -= 1

    def say_hi(self):
        print(self.name, ', says "hi!"')

    def __set_name(self, name):
        if name:
            self.__name = name
        else:
            self.__name = 'Unknown'

    def __get_name(self):
        return self.__name

    @staticmethod
    def car_count():
        return Car.__count

    @classmethod
    def class_count(cls):
        return cls.__count

    def __str__(self):        
        return 'Car with name ' + self.__get_name()

    def __gt__(self, other):
        return self.name > other.name

    name = property(__get_name, __set_name)

# or
# from Car_8 import Car

### Subclasses

- A subclass shares all the attributes and methods of the superclass, but can choose to override behaviour.
  - Subclasses are also known as child classes or subtypes.
- In Python, a subclass is defined by putting the superclass name in parentheses:
```
class Child(Parent):
    def ...
```


In [ ]:
# Would need the next line in a file-based script
# from Car_8 import Car

class HesitantCar(Car):
    def __init__(self, name = None, times = 1):
        super().__init__(name = name)
        self.__times = times

    def say_hi(self):
        print('"', 'um ...' * self.__times, 'hi!", said', super().name)

    def say_goodbye(self):
        print('"', 'um ...' * self.__times, 'goodbye!", said', super().name)

- This is a subclass of `Car`.
  - Look at line 4, where it says `class HesitantCar(Car)`.
- Subclasses can choose to access members of the superclass using `super().`.
  - Can also access them using `ClassName.`, in which case they must pass `self` explicitly:
    - E.g. `Car.__init__(self, name = name)`
    - This mechanism is less flexible if the class structure might change

In [ ]:
c9 = HesitantCar('Mater')

In [ ]:
c9.say_hi()

In [ ]:
c9.say_goodbye()

- `say_hi()` is overridden in `HesitantCar`
- When searching for methods (and attributes), Python uses the MRO (Method Resolution Order).

In [ ]:
HesitantCar.__mro__

- This shows that Python searches classes in the order `HesitantCar`, `Car` and then `object`.
- `object` is a special built-in class.
  - All classes inherit from `object` implicitly if there is no explicit superclass.
  - `object` implements the default methods that we saw earlier.

In [ ]:
HesitantCar.car_count()

In [ ]:
HesitantCar.class_count()

In [ ]:
Car.car_count()

In [ ]:
Car.class_count()

- Can you see why `car_count()` returned `0` whereas `class_count()` returned `1`?
  - To make it clearer, you can follow through in the debugger, or add an extra line to the definition of `Car.__init__()`.

In [ ]:
class Car:
    """
       This is the base class
    """

    __count = 0

    def __init__(self, name = None):
        self.__set_name(name)
        print(type(self))            # Extra line
        type(self).__count += 1

    def __del__(self):
        type(self).__count -= 1

    def say_hi(self):
        print(self.name, ', says "hi!"')

    def __set_name(self, name):
        if name:
            self.__name = name
        else:
            self.__name = 'Unknown'

    def __get_name(self):
        return self.__name

    @staticmethod
    def car_count():
        return Car.__count

    @classmethod
    def class_count(cls):
        return cls.__count

    def __str__(self):        
        return 'Car with name ' + self.__get_name()

    def __gt__(self, other):
        return self.name > other.name

    name = property(__get_name, __set_name)


class HesitantCar(Car):
    def __init__(self, name = None, times = 1):
        super().__init__(name = name)
        self.__times = times

    def say_hi(self):
        print('"', 'um ...' * self.__times, 'hi!", said', super().name)

    def say_goodbye(self):
        print('"', 'um ...' * self.__times, 'goodbye!", said', super().name)

In [ ]:
r10 = HesitantCar('Mater')

- Even though we added that `print` call to the parent class (`Car`), when it was executed, `self` was of type `HesitantCar`. That means the count that was updated in the `__init__()` was associated with `HesitantCar` since it was written `type(self).__count += 1`.
- When we execute `car_count()`, we get the count associated with the superclass, `Car`, not the subclass (hardcoded in the method).
- When we execute `class_count()`, we access the count through the class variable passed in, which will be `HesitantCar`.

This is the essential difference between `@staticmethod` and `@classmethod`, except it is a little simplistic, as we shall see.

### Subclassing Further

Let's create a subclass of `HesitantCar`.

If you created the diagnostic classes above, go back to the start of the notebook and re-define `Car` and `HesitantCar` without the additional diagnostic `print()`.
- It is not enough to recreate just `Car`. Can you see why not?

In [ ]:
# from HesitantCar import HesitantCar

class OldCar(HesitantCar):
    def __init__(self, name = None, times = 3, age = 0):
        super().__init__(name = name, times = times)
        self.__set_age(age)

    def __set_age(self, age):
        if age < 70:
            self.__age = 70
        else:
            self.__age = age

    def __get_age(self):
        return self.__age

    age = property(__get_age, __set_age)

# or
# from OldCar import OldCar

In [ ]:
OldCar.__mro__

In [ ]:
o1 = OldCar(name = 'Chitty Chitty Bang Bang', times = 2, age = 30)

In [ ]:
o1.name

In [ ]:
o1.age

In [ ]:
o1.say_hi()

In [ ]:
OldCar.car_count()

In [ ]:
OldCar.class_count()

Well, that was a little surprising!

There is only one `OldCar`, but the count is `2`. Can you work out why?

The following commands may help you to formulate an opinion.

In [ ]:
OldCar.__dict__

In [ ]:
HesitantCar.__dict__

- `__count` is defined in the scope of `Car`.
  - Python represents this by giving all subclasses a member called `_Car__count`.
  - This is not the same variable (hopefully that much is obvious, because they don't have the same value, but if you want, you can execute `id(OldCar._car__count)` etc to have a look).
- What do you think happens when `OldCar` is defined as a subclass of `HesitantCar`? Remember that a class is really just a dictionary.
  - The dictionary of the parent is copied and new members are added and old members are overridden.
  - It does _not_ refer to the same variables as the superclass, but it _does_ get initialised with the same values.
- So, when `OldCar` is defined, the value of `HesitantCar._Car__count` was already set to 1.
  - This is not an issue with instance variables
  - And not an issue with class hierarchies that are defined before any instances are created, or where no superclass instances are ever created.

If you are still uncertain, take a look at the following example (if you need to, vary the commands to illustrate all situations):

In [ ]:
class Grandparent():
    __count = 0

    def __init__(self):
        type(self).__count += 1

    def __del__(self):
        type(self).__count -= 1

    def __str__(self):
        return '%s: %s' % (type(self), type(self).__count)

class Parent(Grandparent):
    def __init__(self):
        super().__init__()

class Child1(Parent):
    def __init__(self):
        super().__init__()

class Child2(Parent):
    def __init__(self):
        super().__init__()

In [ ]:
c1 = Child1()
p = Parent()
g = Grandparent()

In [ ]:
print(c1)
print(p)
print(g)

In [ ]:
c2 = Child2()
print(c2)

`c1` is created before any instance of the `Parent` or `Grandparent` is created. `Parent._Grandparent__count` has the value `0`. But, by the time `c2` is created, it has the value `1`.

In [ ]:
c1x = Child1()
px = Parent()
gx = Grandparent()
c2x = Child2()
print(c1x)
print(px)
print(gx)
print(c2x)

Once the class object has been created, it counts as you would expect.

## Common Patterns in Inheritance

There is one very common pattern in inheritance. That is to have the class `__init__()` functions defined to accept keyword parameters. The parameters that will be bound in that class are named keywords, and the rest are covered by a `**kwds` parameter to be passed on to the parent.

If we apply this pattern to our classes, we might end up with this:

In [ ]:
class Car:
    """
       This is the base class
    """

    __count = 0

    def __init__(self, name = None):
        self.__set_name(name)
        type(self).__count += 1

    def __del__(self):
        type(self).__count -= 1

    def say_hi(self):
        print(self.name, ', says "hi!"')

    def __set_name(self, name):
        if name:
            self.__name = name
        else:
            self.__name = 'Unknown'

    def __get_name(self):
        return self.__name

    @staticmethod
    def car_count():
        return Car.__count

    @classmethod
    def class_count(cls):
        return cls.__count

    def __str__(self):        
        return 'Car with name ' + self.__get_name()

    def __gt__(self, other):
        return self.name > other.name

    name = property(__get_name, __set_name)

class HesitantCar(Car):
    def __init__(self, times = 1, **kwds):
        super().__init__(**kwds)
        self.__times = times

    def say_hi(self):
        print('"', 'um ...' * self.__times, 'hi!", said', super().name)

    def say_goodbye(self):
        print('"', 'um ...' * self.__times, 'goodbye!", said', super().name)

class OldCar(HesitantCar):
    def __init__(self, age = 0, **kwds):
        super().__init__(**kwds)
        self.__set_age(age)

    def __set_age(self, age):
        if age < 70:
            self.__age = 70
        else:
            self.__age = age

    def __get_age(self):
        return self.__age

    age = property(__get_age, __set_age)


There are some limitations:
- All parameters to the constructors of subclasses must be named
  - Can no longer write `c10 = HesitantCar('Mater')`
- It is not easy to redefine default values in the subclasses
  - We used to do that in `OldCar`, but no longer do
    - It is not impossible, but a little less convenient

In [ ]:
mater = HesitantCar(name = 'Mater')
chitty = OldCar(name = 'Chitty Chitty Bang Bang', times = 2, age = 30)
print(mater)
print(chitty)

## Multiple Inheritance

Python supports multiple inheritance using the following syntax:
```
class SubClassName(Parent1, Parent2):
```
The MRO checks `Parent1` first and then `Parent2`.

This is beyond the scope of this course.

# End of Notebook